# AirBnB Price Prediction Model

This notebook goes over building a machine learning model to try to predict the price of an Airbnb listing in NYC based on a dataset of approximately 50,000 listings.

This report contains several sections
1. [Data Preparation](#Data-Preparation)
2. [Preprocessing and Transformation Pipelines](#Preprocessing-and-Transformation-Pipelines)
3. [Dummy Regressor Model](#Dummy-Regressor)
4. [KNN Regression Model](#KNN-Regressor)
5. [Ridge Regression Model](#Ridge-Regressor)
6. [Decision Tree Regression Model](#Decision-Tree-Regressor)
7. [Alternate Model Using Price Categories](#Binning-Price-and-Using-a-Classification-Model)
8. [Random Forest Regressor](#Random-Forest-Regressor)

The various models were built to try to find the optimal algorithm to use for this dataset.

## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
airbnb=pd.read_csv("AB_NYC_2019.csv")
airbnb.drop(['id','host_id','host_name','last_review'], axis=1, inplace=True)
airbnb['name'] = airbnb['name'].fillna('missing') # could not get simpleimputer to work in pipeline with count vectorizer This seems to be an issue with sklearn 
#https://stackoverflow.com/questions/63000388/how-to-include-simpleimputer-before-countvectorizer-in-a-scikit-learn-pipeline

In [ ]:
airbnb.info()

In [ ]:
airbnb.isnull().sum() #how many nulls are found in each column in dataset 

In [ ]:
# Creating a copy for Omitting the outliers
df = pd.DataFrame()
df = airbnb
df.head()

In [ ]:
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import autocorrelation_plot

#Configure Visualization Defaults
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [ ]:
plt.figure(figsize = (15, 10))
plt.style.use('seaborn-white')

ax = plt.subplot(121)
plt.boxplot(df['price'])
ax.set_title('Price')

ax=plt.subplot(122)
plt.boxplot(df['number_of_reviews'])
ax.set_title('Numer of Reviews')

In [ ]:
# Removing the outliers
# REF: https://towardsdatascience.com/practical-implementation-of-outlier-detection-in-python-90680453b3ce

# 1st & 3rd Quartiles for Price
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)

# Calculating the IQR for Price Column
iqr = q3 - q1 

df_price = df[~((df['price']<(q1-1.5*iqr)) | (df['price']>(q3+1.5*iqr)))]


# 1st & 3rd Quartiles for Number of Reviews
q1 = df_price['number_of_reviews'].quantile(0.25)
q3 = df_price['number_of_reviews'].quantile(0.75)

# Calculating the IQR for Number of Reviews Column
iqr = q3 - q1 

df_new = df_price[~((df_price['number_of_reviews']<(q1-1.5*iqr)) | (df_price['number_of_reviews']>(q3+1.5*iqr)))]

In [ ]:
plt.figure(figsize = (15, 10))
plt.style.use('seaborn-white')

ax = plt.subplot(121)
plt.boxplot(df_new['price'])
ax.set_title('Price')

ax=plt.subplot(122)
plt.boxplot(df_new['number_of_reviews'])
ax.set_title('Numer of Reviews')

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_new, test_size=0.2, random_state=123)

### Investigating the correlation between features

In [ ]:
#Correlation Matrix for numerical values, low correlation between features and target variable likely means it will be difficult to get a high scoring model
#Code sourced from https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas
f= plt.figure(figsize=(19, 15))
plt.matshow(train_df.corr(), fignum=f.number)
plt.xticks(range(train_df.select_dtypes(['number']).shape[1]), train_df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(train_df.select_dtypes(['number']).shape[1]), train_df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
#correlation matrix
train_df.corr()

In [ ]:
# Numerical description
train_df.describe(include=[np.number])

In [ ]:
# Categorical
train_df.describe(include=['O'])

In [ ]:
train_df.groupby('neighbourhood_group').median()[['price','minimum_nights']]

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()  # Allows us to plot big datasets

#Plot histogram
alt.Chart(train_df).mark_bar(
    opacity=0.5,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('neighbourhood_group:N'))

In [ ]:
#create df with lower prices
lowerprices = train_df[train_df['price'] <1000]

In [ ]:
alt.Chart(lowerprices).mark_bar(
    opacity=0.6,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('neighbourhood_group:N'))

In [ ]:
alt.Chart(lowerprices).mark_bar(
    opacity=0.6,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('room_type:N'))

In [ ]:
alt.Chart(lowerprices).mark_bar(
    opacity=0.6,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('number_of_reviews:Q',bin = True))

In [ ]:
alt.Chart(lowerprices).mark_bar(
    opacity=0.6,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('reviews_per_month:Q',bin = True))

In [ ]:
#Function to bin the listings by price according to quartile
def pricerange(price):
    if price <=69:
        return 'low'
    elif price > 69 and price <=106:
        return 'medium-low'
    elif price > 106 and price <=175:
        return 'medium-high'
    elif price >175:
        return 'high'

In [ ]:
#Create a new column in the dataframe that is a price category
train_df['price_range'] = airbnb.apply(lambda x: pricerange(x['price']),axis = 1)

In [ ]:
alt.Chart(train_df).mark_bar(
    opacity=1,
    binSpacing=0
).encode(
    alt.X('price:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('price_range:N',scale=alt.Scale(
            domain=['high', 'medium-high','medium-low','low'],
            range=['black', 'blue','green','red'])))

In [ ]:
#REALLY SLOW
#interactive map slow to load as each point is added to map
#code is taken from a previous project
# create map
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

map_airbnb = folium.Map(location=[40.73293,-73.99782], zoom_start=11)
pricedict = {'low':0,'medium-low':1,'medium-high':2, 'high':3}
colorlist = ['#FF0000','#00FF00','#0000FF','#000000']

# add markers to the map
markers_colors = []
for lat, lon, name, price in zip(train_df['latitude'], train_df['longitude'], train_df['name'], train_df['price_range']):
    label = folium.Popup(str(name) + ' PRICE RANGE: ' + str(price), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=colorlist[pricedict[price]],
        fill=True,
        fill_color=colorlist[pricedict[price]],
        fill_opacity=0.4).add_to(map_airbnb)

In [ ]:
#save the map as html file
map_airbnb
output_file = "nycmap.html"
map_airbnb.save(output_file)

In [ ]:
# Split the data into x,y sets
X_train = train_df.drop(columns = ['price'])
y_train = train_df['price']

X_test = test_df.drop(columns = ['price'])
y_test = test_df['price']

## Preprocessing and Transformation Pipelines

In [ ]:
numeric_features = ['latitude', 'longitude', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count']
categorical_features = ['room_type']
#text_features = ['name']

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer


numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant",fill_value = 0)),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer= Pipeline(steps=[
                ('onehotencoder', OneHotEncoder(handle_unknown="ignore"))])

#text_transformer = make_pipeline(CountVectorizer(max_features = 3),StandardScaler(with_mean=False)) 

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features)
        #("text", text_transformer, 'name')
       
    ], 
    remainder='drop'    
)

## Dummy Regressor

In [ ]:
from sklearn.dummy import DummyRegressor

pipe1 = make_pipeline(
    preprocessor,DummyRegressor())
pipe1

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "dummyregressor__strategy": ['mean']
 
}
random_search = RandomizedSearchCV(pipe1, param_distributions= param_grid, cv=2, verbose=2, n_jobs= 1, n_iter=5, return_train_score = True)
random_search.fit(X_train, y_train);

In [ ]:
pd.DataFrame(random_search.cv_results_)[["params", "mean_test_score","mean_train_score"  , "rank_test_score"]]

## KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

pipe = make_pipeline(
    preprocessor,KNeighborsRegressor())
pipe

In [ ]:
param_grid = {
    "kneighborsregressor__n_neighbors": [30,35,40,50,60]
    
 
}
random_search = RandomizedSearchCV(pipe, param_distributions= param_grid, cv=3, verbose=2, n_jobs=1, n_iter=16, return_train_score = True)
random_search.fit(X_train, y_train);

In [ ]:
from IPython.display import HTML
HTML(pd.DataFrame(random_search.cv_results_)[["params", "mean_test_score", "mean_train_score" ]].to_html())

In [ ]:
random_search.score(X_test,y_test)

## Ridge Regressor

In [ ]:
from sklearn.linear_model import Ridge

pipe3 = make_pipeline(
    preprocessor,Ridge())
pipe3

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    "ridge__alpha": 10.0**np.arange(-2, 4, 1)
 
}
random_search = RandomizedSearchCV(pipe3, param_distributions= param_grid, cv=2, verbose=2, n_jobs= 1, n_iter=6, return_train_score = True)
random_search.fit(X_train, y_train);

In [ ]:
pd.DataFrame(random_search.cv_results_)[["params", "mean_test_score","mean_train_score"  , "rank_test_score"]]

## Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
pipe4 = make_pipeline(
    preprocessor,DecisionTreeRegressor())
pipe4

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "decisiontreeregressor__max_depth": [2,3,4,5]
 
}
random_search = RandomizedSearchCV(pipe4,  param_distributions= param_grid, cv=3, verbose=2, n_jobs=1, n_iter=10, return_train_score = True)
random_search.fit(X_train, y_train);

In [ ]:
pd.DataFrame(random_search.cv_results_)[["params", "mean_test_score", "mean_train_score", "rank_test_score"]]

## Dummy Classifier Model

In [ ]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy = 'most_frequent')
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

## KNeighbours Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipe6 = make_pipeline(
    preprocessor,KNeighborsClassifier())
pipe6

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "kneighborsclassifier__n_neighbors": [5,10,20,30,40,50,60,70,80,90,100]
}

random_search = RandomizedSearchCV(pipe6, param_distributions= param_grid, cv=3, verbose=2, n_jobs=1, n_iter=11, return_train_score = True)
random_search.fit(X_train, y_train);

In [ ]:
pd.DataFrame(random_search.cv_results_)[["params", "mean_test_score", "mean_train_score", "rank_test_score"]]

## Random Forest Regressor

In [ ]:
numeric_features = ['latitude', 'longitude', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count']
categorical_features = ['room_type']

In [ ]:
numeric_transformer = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy = "constant")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(steps=[
                ('onehotencoder', OneHotEncoder(handle_unknown = "ignore"))])

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features)
    ], 
    remainder='drop'    
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
# REF: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 8)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
param_grid = {'rf__n_estimators': n_estimators,
               'rf__max_features': max_features,
               'rf__max_depth': max_depth,
               'rf__min_samples_split': min_samples_split,
               'rf__min_samples_leaf': min_samples_leaf,
               'rf__bootstrap': bootstrap}
pprint(param_grid)

In [ ]:
pipe_rf = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("rf", RandomForestRegressor())])
pipe_rf

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Random search of parameters, using 3 fold cross validation, 
# search across 10 different combinations, and use all available cores
rf_random = RandomizedSearchCV(pipe_rf, param_distributions = param_grid, n_iter = 10, cv = 3, verbose = 2, random_state = 123, n_jobs = -1, return_train_score = True)

# Fit the random search model
rf_random.fit(X_train, y_train);

In [ ]:
rf_random.best_params_

In [ ]:
pd.DataFrame(rf_random.cv_results_)[["params", "mean_test_score", "mean_train_score", "rank_test_score"]]

In [ ]:
rf_random.score(X_test,y_test)

## Binning Price and Using a Classification Model


Due to the low accuracy scores from the regression modeling, we adjusted the model to try to predict the price range that a listing was in by binning the prices into four categories corresponding to approximately the four price quartiles: low, medium-low, medium-high and high.

In [ ]:
#Re-read in the data to start fresh
airbnb=pd.read_csv("AB_NYC_2019.csv")
airbnb.drop(['id','host_id','host_name','last_review'], axis=1, inplace=True)
airbnb['name'] = airbnb['name'].fillna('missing') # could not get simpleimputer to work in pipeline with count vectorizer This seems to be an issue with sklearn 
#https://stackoverflow.com/questions/63000388/how-to-include-simpleimputer-before-countvectorizer-in-a-scikit-learn-pipeline

In [ ]:
def pricerange(price):
    if price <=69:
        return 'low'
    elif price > 69 and price <=106:
        return 'medium-low'
    elif price > 106 and price <=175:
        return 'medium-high'
    elif price >175:
        return 'high'

In [ ]:
airbnb['price_range'] = airbnb.apply(lambda x: pricerange(x['price']),axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(airbnb, test_size=0.2, random_state=123)

In [ ]:
X_train = train_df.drop(columns = ['price', 'price_range'])
y_train = train_df["price_range"]

X_test = test_df.drop(columns=["price","price_range"])
y_test = test_df["price_range"]

In [ ]:
numeric_features = ['latitude', 'longitude', 'minimum_nights' , 'number_of_reviews', 'reviews_per_month','availability_365']
categorical_features = ['room_type']
text_features = ['name']

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer


numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant",fill_value = 0)),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer= Pipeline(steps=[
                ('onehotencoder', OneHotEncoder(handle_unknown="ignore"))])

text_transformer = make_pipeline(CountVectorizer(max_features=3), StandardScaler(with_mean = False)) 

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features),
        ("text", text_transformer, 'name')
       
    ], 
    remainder='drop'    
)